In [25]:
import os
import cohere
import hnswlib
import json
import uuid
from typing import List, Dict
from unstructured.partition.text import partition_text
from unstructured.chunking.title import chunk_by_title

# co = cohere.Client("COHERE_API_KEY")

In [4]:
from dotenv import load_dotenv
load_dotenv()
COHERE_API_KEY=os.getenv('COHERE_API_KEY')

In [5]:
co = cohere.Client(COHERE_API_KEY)

In [6]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [37]:
class Datastore:
    """
    A class representing a collection of documents.

    Parameters:
    sources (list): A list of dictionaries representing the sources of the documents. Each dictionary should have 'title' and 'url' keys.

    Attributes:
    sources (list): A list of dictionaries representing the sources of the documents.
    docs (list): A list of dictionaries representing the documents, with 'title', 'content', and 'url' keys.
    docs_embs (list): A list of the associated embeddings for the documents.
    docs_len (int): The number of documents in the collection.
    index (hnswlib.Index): The index used for document retrieval.

    Methods:
    load_and_chunk(): Loads the data from the sources and partitions the HTML content into chunks.
    embed(): Embeds the documents using the Cohere API.
    index(): Indexes the documents for efficient retrieval.
    """

    def __init__(self, raw_documents: List[Dict[str, str]]):
        self.raw_documents = raw_documents  # raw documents
        self.chunks = []            # chunked version of documents
        self.chunks_embs = []       # embeddings of chunked documents
        self.retrieve_top_k = 10
        self.rerank_top_k = 3
        self.load_and_chunk()  # load raw documents and break into chunks
        self.embed() # generate embeddings for each chunk
        self.index() # store embeddings in an index


    def load_and_chunk(self) -> None:
        """
        Loads the text from the sources and chunks the HTML content.
        """
        print("Loading documents...")

        for source in self.raw_documents:
            elements = partition_text(filename=source["filename"])
            chunks = chunk_by_title(elements)
            for chunk in chunks:
                self.chunks.append(
                    {
                        "title": source["title"],
                        "text": str(chunk),
                        "url": source["filename"],
                    }
                )

    def embed(self) -> None:
        """
        Embeds the document chunks using the Cohere API.
        """
        print("Embedding document chunks...")

        batch_size = 90
        self.chunks_len = len(self.chunks)

        for i in range(0, self.chunks_len, batch_size):
            batch = self.chunks[i : min(i + batch_size, self.chunks_len)]
            texts = [item["text"] for item in batch]
            chunks_embs_batch = co.embed(
                texts=texts, model="embed-english-v3.0", input_type="search_document"
            ).embeddings
            self.chunks_embs.extend(chunks_embs_batch)

    def index(self) -> None:
        """
        Indexes the document chunks for efficient retrieval.
        """
        print("Indexing documents...")

        self.idx = hnswlib.Index(space="ip", dim=1024)
        self.idx.init_index(max_elements=self.chunks_len, ef_construction=512, M=64)
        self.idx.add_items(self.chunks_embs, list(range(len(self.chunks_embs))))

        print(f"Indexing complete with {self.idx.get_current_count()} documents.")

        return self.idx

    def search_and_rerank(self, query: str) -> List[Dict[str, str]]:
        # SEARCH
        query_emb = co.embed(
                  texts=[query], model="embed-english-v3.0", input_type="search_query"
              ).embeddings

        chunk_ids = self.idx.knn_query(query_emb, k=self.retrieve_top_k)[0][0]

        # RERANK
        chunks_to_rerank = [self.chunks[chunk_id]["text"] for chunk_id in chunk_ids]

        rerank_results = co.rerank(
            query=query,
            documents=chunks_to_rerank,
            top_n=self.rerank_top_k,
            model="rerank-english-v2.0",
        )

        chunk_ids_reranked = [chunk_ids[result.index] for result in rerank_results]

        chunks_retrieved = []
        for chunk_id in chunk_ids_reranked:
            chunks_retrieved.append(
                {
                "title": self.chunks[chunk_id]["title"],
                "text": self.chunks[chunk_id]["text"],
                "filename": self.chunks[chunk_id]["url"],
                }
            )

        return chunks_retrieved

In [51]:
sources = [
    {
        "title": "Data Quarkle", 
        "filename": "/home/notebook-user/response-eval-ai/sample-docs/concept-dataquarkle.md"},
    {
        "title": "RAGs", 
        "filename": "/home/notebook-user/response-eval-ai/sample-docs/undestanding-rags.md"},
        {
        "title": "sample", 
        "filename": "/home/notebook-user/response-eval-ai/sample-docs/sample.md"},    
]

# Create an instance of the Datastore class with the given sources
datastore = Datastore(sources)

Loading documents...
Embedding document chunks...
Indexing documents...
Indexing complete with 16 documents.


In [36]:
partition_text(filename='/home/notebook-user/response-eval-ai/sample-docs/concept-dataquarkle.md')

In [54]:
datastore.search_and_rerank("quarkle")

[{'title': 'Data Quarkle',
  'text': 'Introducing "DataQuarkle" – a fantastical approach to data engineering that transcends conventional boundaries and unleashes the power of cosmic data manipulation.',
  'filename': '/home/notebook-user/response-eval-ai/sample-docs/concept-dataquarkle.md'},
 {'title': 'Data Quarkle',
  'text': 'In the realm of DataQuarkle, innovation knows no bounds. From "Datafusion" spells that merge disparate data sources into a harmonious symphony of insights to "Datamancy" rituals that foresee future trends through the cosmic currents, DataQuarkle pushes the boundaries of what is possible in the realm of data engineering.',
  'filename': '/home/notebook-user/response-eval-ai/sample-docs/concept-dataquarkle.md'},
 {'title': 'Data Quarkle',
  'text': 'DataQuarkle harnesses the mysterious energy of "Gigavortex," a celestial phenomenon that swirls through the cosmos, weaving together streams of data from across the universe. Powered by "Quarkblasters," quantum-enhan

In [47]:
class Chatbot:
    def __init__(self, datastore: Datastore):
        """
        Initializes an instance of the Chatbot class.

        Parameters:
        storage (Storage): An instance of the Storage class.

        """
        self.datastore = datastore
        self.conversation_id = str(uuid.uuid4())

    def run(self):
        """
        Runs the chatbot application.

        """
        while True:
            # Get the user message
            message = input("User: ")

            # Typing "quit" ends the conversation
            if message.lower() == "quit":
                print("Ending chat.")
                break
            else:
                print(f"User: {message}")

            # Generate search queries, if any
            response_queries = co.chat(message=message, search_queries_only=True)

            if response_queries.search_queries:
                print("Retrieving information...", end="")

                # Get the query(s)
                queries = []
                for search_query in response_queries.search_queries:
                    queries.append(search_query["text"])

                # Retrieve documents for each query
                chunks = []
                for query in queries:
                    chunks.extend(self.datastore.search_and_rerank(query))
            
                response = co.chat(
                    message=message,
                    documents=chunks,
                    conversation_id=self.conversation_id,
                    stream=True,
                )

            else:
                response = co.chat(
                    message=message,
                    conversation_id=self.conversation_id,
                    stream=True,
                )

            # Print the chatbot response
            print("\nChatbot:")
            
            citations_flag = False
            
            for event in response:
                                
                # Text
                if event.event_type == "text-generation":
                    print(event.text, end="")

                # Citations
                if event.event_type == "citation-generation":
                    if not citations_flag:
                        print("\n\nCITATIONS:")
                        citations_flag = True
                    print(event.citations[0])
            
            # Documents
            if citations_flag:
                print("\n\nDOCUMENTS:")
                documents = [{'id': doc['id'],
                                'text': doc['text'][:50] + '...',
                                'title': doc['title'],
                                'url': doc['filename']} 
                                for doc in response.documents]
                for doc in documents:
                    print(doc)

            print(f"\n{'-'*100}\n")

In [55]:
# Create an instance of the Chatbot class with the Datastore instance
chatbot = Chatbot(datastore)

# Run the chatbot
chatbot.run()

User: data quarkle
Retrieving information...
Chatbot:
DataQuarkle is an innovative approach to data engineering that leverages the power of cosmic data manipulation. It harnesses the energy of a celestial phenomenon called "Gigavortex" and channels it through "Quarkblasters"—quantum-enhanced data processors. This ground-breaking method explores the realm of "Datastral Travel", enabling data engineers to traverse the Dataverse with unprecedented speed and efficiency. 

DataQuarkle also introduces exciting techniques like "Datafusion" spells, which merge diverse data sources into a harmonious symphony of insights, and "Datamancy" rituals that utilise cosmic currents to predict future trends. With DataQuarkle, the possibilities for data engineering are limitless, opening up a new dimension in the understanding and manipulation of data.

CITATIONS:
{'start': 18, 'end': 37, 'text': 'innovative approach', 'document_ids': ['doc_0', 'doc_2']}
{'start': 41, 'end': 57, 'text': 'data engineering'

KeyboardInterrupt: Interrupted by user